Load data

In [2]:
import pandas as pd
import glob
import os

path = "scale_data/scaledata/Dennis+Schwartz/"

all_files = glob.glob(path + "*")

def read_csv(f):
    return pd.read_csv(f, engine="python", delimiter="impossible_delimiter_to_read_line_by_line", names=[os.path.basename(f).rsplit(".", 1)[0]])

def reset_index(df):
    df.reset_index(drop=True, inplace=True)

dataframes = [read_csv(f) for f in all_files]

df = pd.concat(dataframes, axis=1)


df.head()


,id,label.3class,label.4class,rating,subj
0,29420,0,0,0.1,"in my opinion , a movie reviewer's most import..."
1,17219,0,0,0.2,"you can watch this movie , that is based on a ..."
2,18406,0,0,0.2,"this is asking a lot to believe , and though i..."
3,18648,0,0,0.2,no heroes and no story are the main attributes...
4,20021,0,0,0.2,"this is not an art movie , yet i saw it an art..."


In [6]:
df.groupby('label.3class').describe()

id                                                        \
              count          mean          std      min      25%      50%   
label.3class                                                                
0             359.0  25141.816156  3526.028224  17185.0  22774.5  25672.0   
1             427.0  24905.686183  3754.061146  17110.0  22327.5  25908.0   
2             241.0  23518.099585  3896.004687  16748.0  20107.0  23281.0   

                               label.4class            ...           rating  \
                  75%      max        count      mean  ...  75%  max  count   
label.3class                                           ...                    
0             28308.5  29854.0        359.0  0.523677  ...  1.0  1.0  359.0   
1             28063.0  29855.0        427.0  1.409836  ...  2.0  2.0  427.0   
2             26851.0  29812.0        241.0  2.473029  ...  3.0  3.0  241.0   

                                                           
                  mean       std  min  25%  50%  75%  max  
label.3class                                               
0             0.342618  0.066810  0.1  0.3  0.4  0.4  0.4  
1             0.540984  0.049238  0.5  0.5  0.5  0.6  0.6  
2             0.757676  0.067339  0.7  0.7  0.7  0.8  0.9  

[3 rows x 24 columns]

In [5]:
df.groupby('label.4class').describe()

id                                                         \
              count          mean          std      min       25%      50%   
label.4class                                                                 
0             171.0  25320.052632  3509.470790  17185.0  23273.00  26081.0   
1             440.0  24950.111364  3601.047881  17110.0  22505.75  25429.5   
2             302.0  24674.559603  3875.862492  16748.0  21810.00  25855.0   
3             114.0  22535.149123  3746.746965  17109.0  19552.75  22356.0   

                                label.3class            ...           rating  \
                   75%      max        count      mean  ...  75%  max  count   
label.4class                                            ...                    
0             28336.00  29853.0        171.0  0.000000  ...  0.0  0.0  171.0   
1             28124.25  29855.0        440.0  0.572727  ...  1.0  1.0  440.0   
2             28036.50  29814.0        302.0  1.420530  ...  2.0  2.0  302.0   
3             25541.25  29812.0        114.0  2.000000  ...  2.0  2.0  114.0   

                                                           
                  mean       std  min  25%  50%  75%  max  
label.4class                                               
0             0.279532  0.041894  0.1  0.3  0.3  0.3  0.3  
1             0.457273  0.049525  0.4  0.4  0.5  0.5  0.5  
2             0.642053  0.049446  0.6  0.6  0.6  0.7  0.7  
3             0.821930  0.041560  0.8  0.8  0.8  0.8  0.9  

[4 rows x 24 columns]

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.subj, df['label.3class'])

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values)
X_train_count.toarray()[:3]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [71]:
X_test_count = v.transform(X_test)

In [60]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_count, y_train)

MultinomialNB()

In [62]:
model.predict(X_test_count)

array([1, 1, 1, 2, 1, 0, 0, 0, 1, 2, 0, 0, 2, 0, 2, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 2, 0, 1, 1, 0, 2, 0, 2, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 2, 1, 0, 1, 0, 1, 0, 0, 1, 2, 1, 1, 0, 2, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 2, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       2, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 2, 1, 1, 0, 2,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 2, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 2, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 2, 2, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 0, 0,
       0, 0, 1, 1, 2, 1, 1, 2, 2, 0, 2, 0, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 2, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 1, 1, 0, 1, 2,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1], dtype=int64)

In [45]:
model.score(X_test_count, y_test)

0.6147859922178989

In [69]:
X_test.head()

205    this film's narrative promises much to say abo...
959    his rambling story is a near-masterpiece of vi...
998    a brilliant and moving indie drama filmed on a...
982    dariush mehrjui is one of iran's finest modern...
632    in it he questions american values , suggestin...
Name: subj, dtype: object

In [47]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train_count, y_train)

SVC()

In [70]:
clf.predict(X_test_count)

array([0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 2, 1, 2, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 0, 2, 1, 1,
       1, 1, 2, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 2, 0, 0, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 2, 0, 1, 1,
       1, 0, 0, 1, 1, 2, 0, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 2, 1, 1, 0, 0, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 2, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1], dtype=int64)

In [49]:
clf.score(X_test_count, y_test)

0.490272373540856